# Loading JSON dataset

In [1]:
import json

json_path_12k = 'data/IR_data_news_12k.json'
json_path_5k = 'data/IR_data_news_5k.json'

f = open(json_path_12k)
dataset = json.load(f)
print(len(dataset))

12202


In [2]:
dataset_first_idx = 0
dataset_last_idx = len(dataset)-1

print(dataset['0'])

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا', 'category': 'sports'}


# Preprocessing

In [3]:
!pip install parsivar
!pip install hazm


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


## Normalizer

In [4]:
import pickle
import re
from re import sub
import string

def preprocess(text_doc):
    
    def add_space(text_pattern):
        text_pattern = text_pattern.group() 
        text_pattern = text_pattern.strip(' ') 
        text_pattern = " " + text_pattern + " "
        return text_pattern

    def eliminate_pattern(text_pattern):
        return ""

    # Store original emails and IDs
    emails = re.findall(r'\S+@\S+', text_doc)
    ids = re.findall(r'\b(?:[A-Za-z]{2}\d{6}|\d{9})\b', text_doc)

    # Replace emails and IDs with placeholders
    text_doc = re.sub(r'\S+@\S+', 'EMAILADDRESS', text_doc)
    text_doc = re.sub(r'\b(?:[A-Za-z]{2}\d{6}|\d{9})\b', 'IDPLACEHOLDER', text_doc)

    # Handle unwanted characters
    unwanted_chars = r')(}{:؟!،؛»«.' + r"/<>?.,:;"
    unwanted_chars = '[' + unwanted_chars + string.punctuation + ']'
    unwanted_chars = unwanted_chars.replace("@", "")

    # Replace numbers with a placeholder
    pattern = r"[-+]?\d*\.\d+|\d+"
    nums_list = re.findall(pattern, text_doc)
    text_doc = re.sub(pattern, 'floatingpointnumber', text_doc)

    # Add space around unwanted characters
    pattern = '\s*' + unwanted_chars + '+' + '\s*'
    text_doc = re.sub(pattern, add_space, text_doc)

    # Replace newline characters with spaces
    pattern = r'[\n]+'
    text_doc = re.sub(pattern, " ", text_doc)

    # Remove unwanted characters
    unwanted_chars = r")(}{:؟!-،؛»«.@$&%" + r"/<>?.,:;"
    english_chars = r"a-zA-Z0-9"
    pattern = r'[^' + unwanted_chars + english_chars + 'آ-ی' + '‌' + '\d\s:]'
    text_doc = re.sub(pattern, eliminate_pattern, text_doc)

    # Replace multiple spaces with a single space
    pattern = r'[ ]+'
    text_doc = re.sub(pattern, ' ', text_doc)

    # Replace the placeholders with the original numbers
    for number in nums_list:
        text_doc = re.sub('floatingpointnumber', number, text_doc, 1)
        
    # Replace placeholders with original emails and IDs
    for email in emails:
        text_doc = text_doc.replace('EMAILADDRESS', email, 1)
        
    for idx in ids:
        text_doc = text_doc.replace('IDPLACEHOLDER', idx, 1)

    return text_doc.strip()

def unicode_replacement(text_doc):
    replacements = [
        (r"ء", r"ئ"), (r"ٲ|ٱ|إ|ﺍ|أ", r"ا"), (r"ﺁ|آ", r"آ"),
        (r"ﺐ|ﺏ|ﺑ", r"ب"), (r"ﭖ|ﭗ|ﭙ|ﺒ|ﭘ", r"پ"),
        (r"ﭡ|ٺ|ٹ|ﭞ|ٿ|ټ|ﺕ|ﺗ|ﺖ|ﺘ", r"ت"),
        (r"ﺙ|ﺛ", r"ث"), (r"ﺝ|ڃ|ﺠ|ﺟ", r"ج"),
        (r"ڃ|ﭽ|ﭼ", r"چ"), (r"ﺢ|ﺤ|څ|ځ|ﺣ", r"ح"),
        (r"ﺥ|ﺦ|ﺨ|ﺧ", r"خ"), (r"ڏ|ډ|ﺪ|ﺩ", r"د"),
        (r"ﺫ|ﺬ|ﻧ", r"ذ"), (r"ڙ|ڗ|ڒ|ڑ|ڕ|ﺭ|ﺮ", r"ر"),
        (r"ﺰ|ﺯ", r"ز"), (r"ﮊ", r"ژ"), 
        (r"ݭ|ݜ|ﺱ|ﺲ|ښ|ﺴ|ﺳ", r"س"), (r"ﺵ|ﺶ|ﺸ|ﺷ", r"ش"),
        (r"ﺺ|ﺼ|ﺻ", r"ص"), (r"ﺽ|ﺾ|ﺿ|ﻀ", r"ض"),
        (r"ﻁ|ﻂ|ﻃ|ﻄ", r"ط"), (r"ﻆ|ﻇ|ﻈ", r"ظ"),
        (r"ڠ|ﻉ|ﻊ|ﻋ", r"ع"), (r"ﻎ|ۼ|ﻍ|ﻐ|ﻏ", r"غ"),
        (r"ﻒ|ﻑ|ﻔ|ﻓ", r"ف"), (r"ﻕ|ڤ|ﻖ|ﻗ", r"ق"),
        (r"ڭ|ﻚ|ﮎ|ﻜ|ﮏ|ګ|ﻛ|ﮑ|ﮐ|ڪ|ك", r"ک"), 
        (r"ﮚ|ﮒ|ﮓ|ﮕ|ﮔ", r"گ"), 
        (r"ﻝ|ﻞ|ﻠ|ڵ", r"ل"), (r"ﻡ|ﻤ|ﻢ|ﻣ", r"م"),
        (r"ڼ|ﻦ|ﻥ|ﻨ", r"ن"), 
        (r"ވ|ﯙ|ۈ|ۋ|ﺆ|ۊ|ۇ|ۏ|ۅ|ۉ|ﻭ|ﻮ|ؤ", r"و"),
        (r"ﺔ|ﻬ|ھ|ﻩ|ﻫ|ﻪ|ۀ|ە|ة|ہ", r"ه"),
        (r"ﭛ|ﻯ|ۍ|ﻰ|ﻱ|ﻲ|ں|ﻳ|ﻴ|ﯼ|ې|ﯽ|ﯾ|ﯿ|ێ|ے|ى|ي", r"ی")
    ]

    # Process each replacement
    normalized_arabic = text_doc
    for pattern, replacement in replacements:
        text_doc = sub(pattern, replacement, text_doc)
    
    # Normalize numbers
    number_variants = [(r'۰|٠|0', r'۰'), (r'۱|١|1', r'۱'), (r'۲|٢|2', r'۲'), 
                       (r'۳|٣|3', r'۳'), (r'۴|٤|4', r'۴'), (r'۵|5', r'۵'), 
                       (r'۶|٦|6', r'۶'), (r'۷|٧|7', r'۷'), (r'۸|٨|8', r'۸'), 
                       (r'۹|٩|9', r'۹')]

    normalized_numbers = normalized_arabic
    for variant, standard in number_variants:
        normalized_numbers = sub(variant, standard, normalized_numbers)
    
    # Replace unicode symbols
    unicode_replacements = [
        (r'﷽', r'بسم-الله-الرحمن-الرحیم'), (r'ﷻ', r'الله-جل-جلاله'), 
        (r'ﷺ', r'صلی-الله-علیه-وسلم'), (r'ﷲ', r'اله'), 
        (r'ﷳ', r'اکبر'), (r'ﷴ', r'محمد'), (r'ﷵ', r'صلی-الله-علیه-وسلم'), 
        (r'ﷶ', r'رسول'), (r'ﷷ', r'علیه-سلام'), (r'ﷸ|ﷹ', r'صلی-الله-علیه-وسلم'), 
        (r'%|٪', r'درصد')
    ]

    replaced_unicode = normalized_numbers
    for symbol, replacement in unicode_replacements:
        replaced_unicode = sub(symbol, replacement, replaced_unicode)
    
    return replaced_unicode

def unicode_remove(text_doc):
    # Replace various bullet point with a period
    bullet_points = r'•|·|●|·|・|∙|｡|ⴰ'
    replace_with_period = r'.'
    text_with_replaced_bullets = sub(bullet_points, replace_with_period, text_doc)

    # Replace various comma with a standard comma
    comma_variants = r',|٬|٫|‚|，'
    standard_comma = r'،'
    text_with_standard_comma = sub(comma_variants, standard_comma, text_with_replaced_bullets)

    # Remove arabic punctuation characters
    remove_punctuation = r'\ʕ|\.|؛|:|\<|\>|\«|\»|\!|،|\؟'
    text_without_punctuation = sub(remove_punctuation, '', text_with_standard_comma)

    # Remove english punctuation characters
    additional_punctuation = r'\.|\?|\!|\<|\>|\(|\)|;|:'
    text_without_additional_punctuation = sub(additional_punctuation, '', text_without_punctuation)

    # Remove Arabic vowel
    arabic_vowel_marks = r'ـ|ِ|ُ|َ|ٍ|ٌ|ً|'
    text_without_vowel_marks = sub(arabic_vowel_marks, '', text_without_additional_punctuation)

    # Normalize spaces and newlines
    multiple_spaces = r'( )+'
    single_space = r' '
    text_normalized_spaces = sub(multiple_spaces, single_space, text_without_vowel_marks)

    multiple_newlines = r'(\n)+'
    single_newline = r'\n'
    text_normalized_newlines = sub(multiple_newlines, single_newline, text_normalized_spaces)

    return text_normalized_newlines

def space_correction(text_doc):
    # Handle half-space
    start_half_space_pattern = r'^(می|نمی)( )'
    replace_with_half_space = r'\1‌'
    text_with_start_half_space = sub(start_half_space_pattern, replace_with_half_space, text_doc)

    # Add half-space
    middle_half_space_pattern = r'( )(می|نمی)( )'
    replace_middle_with_half_space = r'\1\2‌'
    text_with_middle_half_space = sub(middle_half_space_pattern, replace_middle_with_half_space, text_with_start_half_space)

    # Correct spacing
    suffix_spacing_pattern = r'( )(ی|ای|ها|های|هایی|تر|ترین|گر|گری|ام|ات|اش)( )'
    replace_suffix_spacing = r'‌\2\3'
    corrected_text = sub(suffix_spacing_pattern, replace_suffix_spacing, text_with_middle_half_space)
    
    return corrected_text

def normalize(doc_string):
    normalized_string = unicode_replacement(doc_string)
    normalized_string = unicode_remove(normalized_string)
    normalized_string = preprocess(normalized_string).strip()
    normalized_string = space_correction(normalized_string).strip()

    return normalized_string

## Tokenizer

In [5]:
def tokenize(doc_string):
    token_list = doc_string.strip().split()

    # Half space
    token_list = [token.strip("\u200c") for token in token_list if len(token.strip("\u200c")) != 0]

    return token_list


In [6]:
# # tmp_data = '.می توانید با ایمیل زیر با ما در ارتباط باشید و کیمیا گری کنید bardia@gmail.com'
# # tmp_data = 'سلامي و سلاٍل'
# # tmp_data = 'ارام و آرام و أرام'
# # tmp_data = 'علاعم نگارشی داریم ویرگول، و ؛ و « و » و < و > و ! و ؟ داریم'
# tmp_data = 'بسم الله رو داریم ﷽'

# words = tokenize(normalize(tmp_data))

# words

In [7]:
words = tokenize(normalize(dataset['0']['content']))

words

['به',
 'گزارش',
 'خبرگزاری',
 'فارس',
 'کنفدراسیون',
 'فوتبال',
 'آسیا',
 'AFC',
 'در',
 'نامه\u200cای',
 'رسمی',
 'به',
 'فدراسیون',
 'فوتبال',
 'ایران',
 'و',
 'باشگاه',
 'گیتی',
 'پسند',
 'زمان',
 'قرعه',
 'کشی',
 'جام',
 'باشگاه\u200cهای',
 'فوتسال',
 'آسیا',
 'را',
 'رسما',
 'اعلام',
 'کرد',
 'بر',
 'این',
 'اساس',
 '۲۵',
 'فروردین',
 'ماه',
 '۱۴۰۱',
 'مراسم',
 'قرعه',
 'کشی',
 'جام',
 'باشگاه\u200cهای',
 'فوتسال',
 'آسیا',
 'در',
 'مالزی',
 'برگزار',
 'می\u200cشود',
 'باشگاه',
 'گیتی',
 'پسند',
 'بعنوان',
 'قهرمان',
 'فوتسال',
 'ایران',
 'در',
 'سال',
 '۱۴۰۰',
 'به',
 'این',
 'مسابقات',
 'راه',
 'پیدا',
 'کرده',
 'است',
 'پیش',
 'از',
 'این',
 'گیتی',
 'پسند',
 'تجربه',
 '۳',
 'دوره',
 'حضور',
 'در',
 'جام',
 'باشگاه\u200cهای',
 'فوتسال',
 'آسیا',
 'را',
 'داشته',
 'که',
 'هر',
 'سه',
 'دوره',
 'به',
 'فینال',
 'مسابقات',
 'راه',
 'پیدا',
 'کرده',
 'و',
 'یک',
 'عنوان',
 'قهرمانی',
 'و',
 'دو',
 'مقام',
 'دومی',
 'بدست',
 'آورده',
 'است',
 'انتهای',
 'پیام',
 '/']

## Tokenizing and normalizing the whole data

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def process_content(key):
    words = tokenize(normalize(dataset[key]['content']))
    # words = tokenize(dataset[key]['content'])
    return key, words

tokenized = {}

num_threads = 4

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Create a dictionary of future tasks
    futures = {executor.submit(process_content, key): key for key in dataset}

    # Use tqdm to display progress
    for future in tqdm(as_completed(futures), total=len(dataset), desc="Tokenizing"):
        key, words = future.result()
        tokenized[key] = words

Tokenizing: 100%|██████████| 12202/12202 [00:17<00:00, 679.29it/s] 


In [9]:
total_tokens = sum(len(tokens) for tokens in tokenized.values())

print(f"Total number of tokens in the dataset: {total_tokens}")

Total number of tokens in the dataset: 4554642


In [10]:
print(tokenized['0'])

['به', 'گزارش', 'خبرگزاری', 'فارس', 'کنفدراسیون', 'فوتبال', 'آسیا', 'AFC', 'در', 'نامه\u200cای', 'رسمی', 'به', 'فدراسیون', 'فوتبال', 'ایران', 'و', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه', 'کشی', 'جام', 'باشگاه\u200cهای', 'فوتسال', 'آسیا', 'را', 'رسما', 'اعلام', 'کرد', 'بر', 'این', 'اساس', '۲۵', 'فروردین', 'ماه', '۱۴۰۱', 'مراسم', 'قرعه', 'کشی', 'جام', 'باشگاه\u200cهای', 'فوتسال', 'آسیا', 'در', 'مالزی', 'برگزار', 'می\u200cشود', 'باشگاه', 'گیتی', 'پسند', 'بعنوان', 'قهرمان', 'فوتسال', 'ایران', 'در', 'سال', '۱۴۰۰', 'به', 'این', 'مسابقات', 'راه', 'پیدا', 'کرده', 'است', 'پیش', 'از', 'این', 'گیتی', 'پسند', 'تجربه', '۳', 'دوره', 'حضور', 'در', 'جام', 'باشگاه\u200cهای', 'فوتسال', 'آسیا', 'را', 'داشته', 'که', 'هر', 'سه', 'دوره', 'به', 'فینال', 'مسابقات', 'راه', 'پیدا', 'کرده', 'و', 'یک', 'عنوان', 'قهرمانی', 'و', 'دو', 'مقام', 'دومی', 'بدست', 'آورده', 'است', 'انتهای', 'پیام', '/']


## Remove stopping words

In [11]:
from collections import Counter

def count_words(token_lists):
    word_counter = Counter()
    for tokens in token_lists:
        word_counter.update(tokens)
    return word_counter

def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k: data[k] for k in list(data)[i:i + SIZE]}

chunked_data = list(chunks(tokenized, len(tokenized) // 4))

word_counts = Counter()
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(count_words, chunk.values()): chunk for chunk in chunked_data}
    for future in tqdm(as_completed(futures), total=len(chunked_data), desc="Counting Words"):
        word_counts.update(future.result())

stop_words = set([word for word, count in word_counts.most_common(50)])
print(stop_words)

Counting Words: 100%|██████████| 5/5 [00:00<00:00, 237.42it/s]

{'داشت', 'شد', 'که', 'اسلامی', 'دولت', 'کند', 'مجلس', 'آن', 'انتهای', 'یک', 'با', 'به', 'کرد', 'کشور', 'اما', 'دارد', 'خبرگزاری', 'در', 'باید', 'شده', 'را', 'سال', '/', 'دو', 'است', 'بر', 'این', 'تیم', 'هم', 'ایران', 'تا', 'پیام', 'اینکه', 'رئیس', 'گزارش', 'شود', 'ما', 'قرار', 'برای', 'بازی', 'مردم', 'ملی', '-', 'گفت', 'خود', 'و', 'بود', 'فارس', 'وی', 'از'}


In [12]:
def remove_top_words_and_count(tokens):
    initial_count = len(tokens)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    removed_count = initial_count - len(filtered_tokens)
    return filtered_tokens, removed_count

total_removed_tokens = 0
removed_tokenized = {}

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(remove_top_words_and_count, tokenized[key]): key for key in tokenized}
    for future in tqdm(as_completed(futures), total=len(tokenized), desc="Removing Top Words"):
        result = future.result()
        if isinstance(result, tuple) and len(result) == 2:
            key = futures[future]
            filtered_tokens, removed_count = result
            removed_tokenized[key] = filtered_tokens
            total_removed_tokens += removed_count
        else:
            print(f"Unexpected result format: {result}")

print(f"Top 50 most frequent words removed from tokenized data. Total tokens removed: {total_removed_tokens}")

Removing Top Words: 100%|██████████| 12202/12202 [00:00<00:00, 408897.98it/s]

Top 50 most frequent words removed from tokenized data. Total tokens removed: 1558730


In [13]:
total_tokens_filtered = sum(len(tokens) for tokens in removed_tokenized.values())

print(f"Total number of tokens in the dataset after removing stopping words: {total_tokens_filtered}")

Total number of tokens in the dataset after removing stopping words: 2995912


In [14]:
print(removed_tokenized['0'])

['کنفدراسیون', 'فوتبال', 'آسیا', 'AFC', 'نامه\u200cای', 'رسمی', 'فدراسیون', 'فوتبال', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه', 'کشی', 'جام', 'باشگاه\u200cهای', 'فوتسال', 'آسیا', 'رسما', 'اعلام', 'اساس', '۲۵', 'فروردین', 'ماه', '۱۴۰۱', 'مراسم', 'قرعه', 'کشی', 'جام', 'باشگاه\u200cهای', 'فوتسال', 'آسیا', 'مالزی', 'برگزار', 'می\u200cشود', 'باشگاه', 'گیتی', 'پسند', 'بعنوان', 'قهرمان', 'فوتسال', '۱۴۰۰', 'مسابقات', 'راه', 'پیدا', 'کرده', 'پیش', 'گیتی', 'پسند', 'تجربه', '۳', 'دوره', 'حضور', 'جام', 'باشگاه\u200cهای', 'فوتسال', 'آسیا', 'داشته', 'هر', 'سه', 'دوره', 'فینال', 'مسابقات', 'راه', 'پیدا', 'کرده', 'عنوان', 'قهرمانی', 'مقام', 'دومی', 'بدست', 'آورده']


# Stemming

In [15]:
from parsivar import FindStems

stemmer = FindStems()

def stem_tokens(tokens):
    return [stemmer.convert_to_stem(word) for word in tokens]

stemmed_tokenized = {}

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = {executor.submit(stem_tokens, removed_tokenized[key]): key for key in removed_tokenized}

    for future in tqdm(as_completed(futures), total=len(tokenized), desc="Stemming Tokens"):
        key = futures[future]
        stemmed_tokenized[key] = future.result()

Stemming Tokens: 100%|██████████| 12202/12202 [00:10<00:00, 1153.24it/s]


In [16]:
print(stemmed_tokenized['0'])

['کنفدراسیون', 'فوتبال', 'آسیا', 'AFC', 'نامه', 'رسمی', 'فدراسیون', 'فوتبال', 'باشگاه', 'گیتی', 'پسند', 'زمان', 'قرعه', 'کشید&کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'رسما', 'اعلام', 'اساس', '۲۵', 'فروردین', 'ماه', '۱۴۰۱', 'مراسم', 'قرعه', 'کشید&کش', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'مالزی', 'برگزار', 'شد&شو', 'باشگاه', 'گیتی', 'پسند', 'بعنوان', 'قهرمان', 'فوتسال', '۱۴۰۰', 'مسابقات', 'راه', 'پیدا', 'کرده', 'پیش', 'گیتی', 'پسند', 'تجربه', '۳', 'دوره', 'حضور', 'جام', 'باشگاه', 'فوتسال', 'آسیا', 'داشته', 'هر', 'سه', 'دوره', 'فینال', 'مسابقات', 'راه', 'پیدا', 'کرده', 'عنوان', 'قهرمانی', 'مقام', 'دومی', 'بدست', 'آورده']


# Inverted Positional Index

In [17]:
inverted_index = {}

# Iterate over each document
for doc_id in tqdm(stemmed_tokenized, desc="Processing Documents"):
    # Iterate over each word in the document
    for position, word in enumerate(stemmed_tokenized[doc_id]):
        if word not in inverted_index:
            # Create a new entry for this word if not already present
            doc_dict = {doc_id: (1, [position])}
            inverted_index[word] = (1, doc_dict)
        else:
            # Update the existing entry for the word
            total_freq, doc_dict = inverted_index[word]
            total_freq += 1

            if doc_id in doc_dict:
                # Word is already present in this document
                doc_freq, positions = doc_dict[doc_id]
                doc_freq += 1
                positions.append(position)
                doc_dict[doc_id] = (doc_freq, positions)
            else:
                # First occurrence of this word in this document
                doc_dict[doc_id] = (1, [position])

            # Update the inverted index
            inverted_index[word] = (total_freq, doc_dict)

print(f"Total unique words in the index: {len(inverted_index)}")

Processing Documents: 100%|██████████| 12202/12202 [00:15<00:00, 787.08it/s] 

Total unique words in the index: 50272


In [18]:
inverted_index['امیرکبیر']

(45,
 {'10024': (2, [97, 119]),
  '9845': (1, [1310]),
  '8463': (2, [1097, 1137]),
  '10568': (2, [660, 746]),
  '7416': (1, [22]),
  '10681': (1, [18]),
  '7395': (1, [14]),
  '7579': (1, [983]),
  '11603': (6, [14, 21, 49, 95, 167, 236]),
  '11437': (1, [3528]),
  '10361': (1, [72]),
  '7535': (1, [6]),
  '11176': (1, [118]),
  '7634': (1, [2009]),
  '11694': (1, [263]),
  '10567': (3, [157, 189, 208]),
  '8125': (4, [21, 33, 60, 126]),
  '7346': (2, [7, 96]),
  '8645': (1, [16]),
  '12134': (1, [402]),
  '9808': (1, [68]),
  '11645': (1, [16]),
  '11405': (3, [55, 1400, 1500]),
  '9286': (5, [893, 1081, 1121, 1349, 1365]),
  '9261': (1, [38])})

# Champions list

In [19]:
K_CHAMPION = 20
champion_lists = {}

# Iterate over each word and its inverted index
for word, (_, doc_dict) in tqdm(inverted_index.items(), desc="Building Champion Lists"):
    champions = {}
    # Iterate over each document
    for doc_id, (doc_freq, positions) in doc_dict.items():
        # Build the champion list
        if len(champions) < K_CHAMPION:
            champions[doc_id] = (doc_freq, positions)
        else:
            # Identify the document with the minimum frequency in the current champions; replace if doc_freq is higher
            min_freq_doc_id = min(champions, key=lambda d: champions[d][0])
            if champions[min_freq_doc_id][0] < doc_freq:
                del champions[min_freq_doc_id]
                champions[doc_id] = (doc_freq, positions)

    # Calculate the total frequency within champion documents
    total_freq_in_champions = sum(doc_freq for doc_freq, _ in champions.values())

    # Store the champion list for the word along with its total frequency within these documents
    champion_lists[word] = (total_freq_in_champions, champions)

Building Champion Lists: 100%|██████████| 50272/50272 [00:02<00:00, 17666.46it/s] 


In [20]:
print(champion_lists['امیرکبیر'])

(40, {'10024': (2, [97, 119]), '8463': (2, [1097, 1137]), '10568': (2, [660, 746]), '10681': (1, [18]), '7395': (1, [14]), '7579': (1, [983]), '11603': (6, [14, 21, 49, 95, 167, 236]), '11437': (1, [3528]), '10361': (1, [72]), '7535': (1, [6]), '11176': (1, [118]), '7634': (1, [2009]), '11694': (1, [263]), '10567': (3, [157, 189, 208]), '8125': (4, [21, 33, 60, 126]), '7346': (2, [7, 96]), '8645': (1, [16]), '12134': (1, [402]), '11405': (3, [55, 1400, 1500]), '9286': (5, [893, 1081, 1121, 1349, 1365])})


# Vectorized documents

In [21]:
import math
from collections import defaultdict

def create_vectorized_docs(tokenized_docs):
    doc_freq = defaultdict(int)
    vectorized_docs = {}
    weights_docs = {}

    # Calculate document frequency for each term across all documents
    for tokens in tqdm(tokenized_docs.values(), desc="Calculating Document Frequency"):
        unique_terms = set(tokens)
        for term in unique_terms:
            doc_freq[term] += 1

    N = len(tokenized_docs)

    # TF-IDF
    for doc_id, tokens in tqdm(tokenized_docs.items(), desc="Calculating TF-IDF"):
        term_tf_idf = {}
        term_freq = defaultdict(int)

        # Calculate term frequency for each word in the document
        for word in tokens:
            term_freq[word] += 1
        
        # Calculate TF-IDF
        for word, freq in term_freq.items():
            tf = 1 + math.log(freq, 10)  # Term frequency part of TF-IDF
            idf = math.log(N / doc_freq[word], 10)  # Inverse document frequency part
            tf_idf = tf * idf
            term_tf_idf[word] = tf_idf

#         # Calculate the average term frequency for the document
#         avg_tf = sum(term_freq.values()) / len(term_freq)

#         # Calculate TF-IDF for each term using the new formula
#         for word, freq in term_freq.items():
#             tf = (1 + math.log(freq, 10)) / (1 + math.log(avg_tf, 10))
#             idf = max(0, math.log((N - doc_freq[word]) / doc_freq[word], 10))
#             tf_idf = tf * idf
#             term_tf_idf[word] = tf_idf
            
        # Store the raw TF-IDF weights
        weights_docs[doc_id] = term_tf_idf.copy()
        
        squared_length = sum(tf_idf_val ** 2 for tf_idf_val in term_tf_idf.values())
        
        # Normalize the TF-IDF values
        vectorized_docs[doc_id] = {word: tf_idf / math.sqrt(squared_length) for word, tf_idf in term_tf_idf.items()}
        # vectorized_docs[doc_id] = {word: tf_idf for word, tf_idf in term_tf_idf.items()}


    return vectorized_docs, weights_docs, doc_freq, N

vectorized_docs, weights_docs, doc_freq, N = create_vectorized_docs(stemmed_tokenized)
vectorized_champ, weights_champ, champ_freq, N_champ = create_vectorized_docs(stemmed_tokenized)

Calculating TF-IDF: 100%|██████████| 12202/12202 [00:01<00:00, 9523.84it/s]


In [22]:
print("Normalized TF-IDF Vector:\n", vectorized_docs.get('0', "Document ID '0' not found"), '\n')
print("TF-IDF Weights:\n", weights_docs.get('0', "Document ID '0' not found"))
print(doc_freq['فوتبال'])
print(N)

Normalized TF-IDF Vector:
 {'کنفدراسیون': 0.16850184189944337, 'فوتبال': 0.0703236191265043, 'آسیا': 0.1619234797353218, 'AFC': 0.17117260522029595, 'نامه': 0.10162172546380702, 'رسمی': 0.11444592394013639, 'فدراسیون': 0.08644086684330668, 'باشگاه': 0.11285094997158054, 'گیتی': 0.3398172566260429, 'پسند': 0.31460042764243656, 'زمان': 0.07742081140273063, 'قرعه': 0.25974422601846275, 'کشید&کش': 0.14583186573880313, 'جام': 0.1218245333140782, 'فوتسال': 0.2735522745525411, 'رسما': 0.1825853638583551, 'اعلام': 0.06202257640888991, 'اساس': 0.08746681643165777, '۲۵': 0.12578011653478952, 'فروردین': 0.17203030366445962, 'ماه': 0.07254421163300315, '۱۴۰۱': 0.11531716598400991, 'مراسم': 0.12257691153491386, 'مالزی': 0.2327832350949502, 'برگزار': 0.06015270256259929, 'شد&شو': 0.029175414722865226, 'بعنوان': 0.18578138768492783, 'قهرمان': 0.09773868385894881, '۱۴۰۰': 0.11054095993277666, 'مسابقات': 0.10948184665469728, 'راه': 0.10723681961760784, 'پیدا': 0.10948184665469728, 'کرده': 0.06319339644

In [23]:
print("Normalized TF-IDF Vector:\n", vectorized_champ.get('0', "Document ID '0' not found"), '\n')
print("TF-IDF Weights:\n", weights_champ.get('0', "Document ID '0' not found"))
print(champ_freq['فوتبال'])
print(N_champ)

Normalized TF-IDF Vector:
 {'کنفدراسیون': 0.16850184189944337, 'فوتبال': 0.0703236191265043, 'آسیا': 0.1619234797353218, 'AFC': 0.17117260522029595, 'نامه': 0.10162172546380702, 'رسمی': 0.11444592394013639, 'فدراسیون': 0.08644086684330668, 'باشگاه': 0.11285094997158054, 'گیتی': 0.3398172566260429, 'پسند': 0.31460042764243656, 'زمان': 0.07742081140273063, 'قرعه': 0.25974422601846275, 'کشید&کش': 0.14583186573880313, 'جام': 0.1218245333140782, 'فوتسال': 0.2735522745525411, 'رسما': 0.1825853638583551, 'اعلام': 0.06202257640888991, 'اساس': 0.08746681643165777, '۲۵': 0.12578011653478952, 'فروردین': 0.17203030366445962, 'ماه': 0.07254421163300315, '۱۴۰۱': 0.11531716598400991, 'مراسم': 0.12257691153491386, 'مالزی': 0.2327832350949502, 'برگزار': 0.06015270256259929, 'شد&شو': 0.029175414722865226, 'بعنوان': 0.18578138768492783, 'قهرمان': 0.09773868385894881, '۱۴۰۰': 0.11054095993277666, 'مسابقات': 0.10948184665469728, 'راه': 0.10723681961760784, 'پیدا': 0.10948184665469728, 'کرده': 0.06319339644

# Similarity

In [24]:
def vectorize_query(raw_query, doc_freq, N):
    # Tokenize the query and count term frequencies
    term_freq = defaultdict(int)
    for term in raw_query.split():
        term_freq[term] += 1

    term_tf_idf = {}
    for word, freq in term_freq.items():
        if word in doc_freq:
            tf = 1 + math.log(freq, 10)  # Term frequency in the query
            idf = math.log(N / doc_freq[word], 10)  # Inverse document frequency
            term_tf_idf[word] = tf * idf  # TF-IDF weight for the term in the query
#             term_tf_idf[word] = (tf * idf) * freq  # Amplified TF-IDF weight for frequent terms

    # Normalize the query vector
    query_len = math.sqrt(sum(weight**2 for weight in term_tf_idf.values()))
    return {word: weight / query_len for word, weight in term_tf_idf.items()}
    # return {word: weight for word, weight in term_tf_idf.items()}


raw_query = "فوتبال جام جهانی"
vectorized_query = vectorize_query(raw_query, doc_freq, N)
print(vectorized_query)

{'فوتبال': 0.40808435260589404, 'جام': 0.6226652771934315, 'جهانی': 0.6676489449822082}


## Cosine

In [25]:
def cosine_similarity(query_vector, doc_vector):
    dot_product = sum(query_vector.get(word, 0) * doc_vector.get(word, 0) for word in query_vector)
    return dot_product  # Since both vectors are normalized, no need to divide by magnitudes.

In [26]:
query = 'دانشگاه امیرکبیر'
doc_id = '0'
similarity = cosine_similarity(vectorize_query(query, doc_freq, N), vectorized_docs.get(f'''{doc_id}'''))
print(f"Cosine similarity between the query and document {doc_id}: {similarity}")

Cosine similarity between the query and document 0: 0.0


## Jaccard

In [27]:
def jaccard_similarity(query_set, doc_set):
    intersection = query_set.intersection(doc_set)
    union = query_set.union(doc_set)
    return len(intersection) / len(union)

In [28]:
query = 'دانشگاه امیرکبیر'
doc_id = '0'

doc_terms = stemmed_tokenized[doc_id] if doc_id in stemmed_tokenized else []
doc_set = set(doc_terms)

similarity = jaccard_similarity(set(query.split()), doc_set)
print(f"Jaccard similarity between the query and document {doc_id}: {similarity}")

Jaccard similarity between the query and document 0: 0.0


# Query retrival

In [29]:
stemmer = FindStems()

def stem_tokens(tokens):
    return [stemmer.convert_to_stem(word) for word in tokens]

def preprocess_query(query_text, doc_freq, N):
    # Tokenize, remove stop words, and stem the query
    query_tokens = stem_tokens(remove_top_words_and_count(tokenize(normalize(query_text)))[0])
    
    # Build an inverted positional index for the query
    query_index = {}
    for position, term in enumerate(query_tokens):
        if term not in query_index:
            query_index[term] = [position]
        else:
            query_index[term].append(position)
    
    # Vectorize the query
    vectorized_query = vectorize_query(query_text, doc_freq, N)
    
    return query_index, vectorized_query

query_text = "فوتبال جام جهانی"
query_index, vectorized_query = preprocess_query(query_text, doc_freq, N)
print("Inverted Positional Index for Query:")
print(query_index)
print("\nVectorized Query:")
print(vectorized_query)

Inverted Positional Index for Query:
{'فوتبال': [0], 'جام': [1], 'جهانی': [2]}

Vectorized Query:
{'فوتبال': 0.40808435260589404, 'جام': 0.6226652771934315, 'جهانی': 0.6676489449822082}


In [30]:
import heapq

def find_top_k_documents(query_text, similarity_function, vectorized_docs, doc_freq, N, k=10):
    query_index, vectorized_query = preprocess_query(query_text, doc_freq, N)
    
    top_k_documents = []
    
    # Calculate similarity between the query and each document
    for doc_id in tqdm(stemmed_tokenized, desc="Calculating Similarity"):
        if similarity_function == 'cosine':
            similarity_score = cosine_similarity(vectorized_query, vectorized_docs[doc_id])
        elif similarity_function == 'jaccard':
            doc_terms = stemmed_tokenized[doc_id] if doc_id in stemmed_tokenized else []
            doc_set = set(doc_terms)

            query_set = set(query_index.keys())
            similarity_score = jaccard_similarity(query_set, doc_set)
        else:
            raise ValueError("Invalid similarity function. Use 'cosine' or 'jaccard'.")
        
        heapq.heappush(top_k_documents, (similarity_score, doc_id))
        
        # If the heap size exceeds k, remove the smallest element
        if len(top_k_documents) > k:
            heapq.heappop(top_k_documents)

    top_k_documents = [(score, doc_id) for score, doc_id in top_k_documents]
    
    return sorted(top_k_documents, key=lambda doc: doc[0], reverse=True)

In [31]:
query_text = "کیس های هک شدن دستبرد"
similarity_function = 'cosine'
k = 10
top_k_documents = find_top_k_documents(query_text, similarity_function, vectorized_docs, doc_freq, N, k)
print(f"Top {k} documents most relevant to the query:")
for score, doc_id in top_k_documents:
    print(f"Document ID: {doc_id}, Similarity Score: {score}")

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 364042.38it/s]

Top 10 documents most relevant to the query:
Document ID: 11779, Similarity Score: 0.16123364690526393
Document ID: 4017, Similarity Score: 0.14786926110611576
Document ID: 5156, Similarity Score: 0.11878121035578779
Document ID: 460, Similarity Score: 0.10862863993778082
Document ID: 7498, Similarity Score: 0.1077733598287549
Document ID: 3453, Similarity Score: 0.10039410430573438
Document ID: 10502, Similarity Score: 0.09637041057472998
Document ID: 10744, Similarity Score: 0.0935898245796879
Document ID: 11165, Similarity Score: 0.09202864873110608
Document ID: 11015, Similarity Score: 0.08944875654111803


In [32]:
query_text = "فوتبال جام جهانی"
similarity_function = 'cosine'
k = 10
top_k_documents = find_top_k_documents(query_text, similarity_function, vectorized_champ, champ_freq, N_champ, k)
print(f"Top {k} documents most relevant to the query:")
for score, doc_id in top_k_documents:
    print(f"Document ID: {doc_id}, Similarity Score: {score}")

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 420119.01it/s]

Top 10 documents most relevant to the query:
Document ID: 3512, Similarity Score: 0.2832998395132039
Document ID: 3505, Similarity Score: 0.24858583957413377
Document ID: 3498, Similarity Score: 0.24142371633217855
Document ID: 3511, Similarity Score: 0.24054498731718865
Document ID: 3119, Similarity Score: 0.2379384482711294
Document ID: 3492, Similarity Score: 0.23716938096752493
Document ID: 3529, Similarity Score: 0.2324444209702053
Document ID: 3330, Similarity Score: 0.23086505090324239
Document ID: 3502, Similarity Score: 0.22346650769190957
Document ID: 3496, Similarity Score: 0.2211427345087517


## Show data

In [57]:
import time
import heapq

def show_data(doc_id, query, similarity_score):
    document = dataset[doc_id]
    print(f"Document ID: {doc_id}")
    if similarity_score:
        print(f"Similarity Score: {similarity_score:.4f}")
    print('\033[1m' + document['title'] + '\033[0m')
    print('\033[1m' + document['url']+ '\033[0m')

    content_lines = document['content'].split('\n')

    # Filter lines that contain any of the query words
    filtered_lines = [line for line in content_lines if any(word in line for word in query)]

    # Print the filtered lines
    print("\n".join(filtered_lines[:3]))  # Print the first 3 matching lines
    print()

def query(query_text, similarity_function, k=10, use_champion_list=False):
    # Start the timer
    start_time = time.time()

    top_k_documents = []
    existing_docs = set()

    if use_champion_list:
        # Find top k documents from the champion list
        top_k_documents = find_top_k_documents(query_text, similarity_function, vectorized_champ, champ_freq, N_champ, k)
        existing_docs = set([doc_id for _, doc_id in top_k_documents])

    else:
        top_k_documents = find_top_k_documents(query_text, similarity_function, vectorized_docs, doc_freq, N, k)

    # Check if more documents are needed from the normal list
    if use_champion_list and len(top_k_documents) < k:
        print('needed')
        additional_k = 2 * (k - len(top_k_documents))
        additional_documents = find_top_k_documents(query_text, similarity_function, vectorized_docs, doc_freq, N, additional_k)
        
        # Add unique documents from additional_documents to top_k_documents
        for score, doc_id in additional_documents:
            if doc_id not in existing_docs and len(top_k_documents) < k:
                top_k_documents.append((score, doc_id))
                existing_docs.add(doc_id)

    # End the timer
    end_time = time.time()
    execution_time = end_time - start_time

    # Display the results
    print(f"Execution Time: {execution_time:.4f} seconds")
    print(f"Top {k} documents most relevant to the query:\n")
    for rank, (similarity_score, doc_id) in enumerate(top_k_documents, start=1):
        print(f'''Rank {rank}''')
        show_data(doc_id, query_index.keys(), similarity_score)
        
    return top_k_documents

In [59]:
query_text = "فوتبال"
similarity_function = 'cosine'
k = 10
query1 = query(query_text, similarity_function, k)

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 305454.48it/s]

Execution Time: 0.0433 seconds
Top 10 documents most relevant to the query:

Rank 1
Document ID: 6383
Similarity Score: 0.1348
برگزاری نشست هم اندیشی هیات‌رئیسه فدراسیون و روسای هیات‌های فوتبال در اصفهان
https://www.farsnews.ir/news/14001001000273/برگزاری-نشست-هم-اندیشی-هیات‌رئیسه-فدراسیون-و-روسای-هیات‌های-فوتبال-در
به گزارش خبرنگار ورزشی خبرگزاری فارس، نشست اعضای هیات رئیسه فدراسیون فوتبال و روسای هیات‌های فوتبال کل کشور فردا برگزار می شود. این نشست فردا در اصفهان برگزار خواهد شد و اعضا در مورد مسائل مختلف در فوتبال کشور به بحث و تبادل نظر خواهند پرداخت. قرار است اعضای هیات رئیسه فدراسیون و همچنین شهاب عزیزی خادم امروز راهی اصفهان شوند.  انتهای پیام/

Rank 2
Document ID: 719
Similarity Score: 0.1345
برگزاری مجمع انتخاباتی هیات فوتبال خراسان با حضور اعضای هیات رئیسه فدراسیون
https://www.farsnews.ir/news/14001215000166/برگزاری-مجمع-انتخاباتی-هیات-فوتبال-خراسان-با-حضور-اعضای-هیات-رئیسه
به گزارش خبرنگار ورزشی خبرگزاری فارس، مجمع هیات فوتبال استان خراسان قرار است روز پنجشنبه 19 اسفند در مش

In [60]:
query_text = "فوتبال"
similarity_function = 'cosine'
k = 10
query1 = query(query_text, similarity_function, k, use_champion_list=True)

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 315183.69it/s]

Execution Time: 0.0405 seconds
Top 10 documents most relevant to the query:

Rank 1
Document ID: 6383
Similarity Score: 0.1348
برگزاری نشست هم اندیشی هیات‌رئیسه فدراسیون و روسای هیات‌های فوتبال در اصفهان
https://www.farsnews.ir/news/14001001000273/برگزاری-نشست-هم-اندیشی-هیات‌رئیسه-فدراسیون-و-روسای-هیات‌های-فوتبال-در
به گزارش خبرنگار ورزشی خبرگزاری فارس، نشست اعضای هیات رئیسه فدراسیون فوتبال و روسای هیات‌های فوتبال کل کشور فردا برگزار می شود. این نشست فردا در اصفهان برگزار خواهد شد و اعضا در مورد مسائل مختلف در فوتبال کشور به بحث و تبادل نظر خواهند پرداخت. قرار است اعضای هیات رئیسه فدراسیون و همچنین شهاب عزیزی خادم امروز راهی اصفهان شوند.  انتهای پیام/

Rank 2
Document ID: 719
Similarity Score: 0.1345
برگزاری مجمع انتخاباتی هیات فوتبال خراسان با حضور اعضای هیات رئیسه فدراسیون
https://www.farsnews.ir/news/14001215000166/برگزاری-مجمع-انتخاباتی-هیات-فوتبال-خراسان-با-حضور-اعضای-هیات-رئیسه
به گزارش خبرنگار ورزشی خبرگزاری فارس، مجمع هیات فوتبال استان خراسان قرار است روز پنجشنبه 19 اسفند در مش

In [61]:
query_text = "تیم فوتبال ایران"
similarity_function = 'cosine'
k = 10
query1 = query(query_text, similarity_function, k)

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 266773.51it/s]

Execution Time: 0.0484 seconds
Top 10 documents most relevant to the query:

Rank 1
Document ID: 1143
Similarity Score: 0.1802
واکنش قاسمی به حضور مادرش در سالن: گفته بودم یا می‌برم یا می‌میرم
https://www.farsnews.ir/news/14001208001155/واکنش-قاسمی-به-حضور-مادرش-در-سالن-گفته-بودم-یا-می‌برم-یا-می‌میرم
به گزارش خبرنگار ورزشی خبرگزاری فارس، فینال سنگین وزن کشتی فرنگی جام تختی در حال بین امیرقاسمی و فرنگی‌کار ارمنستان برگزار شد که مادر کشتی‌گیر کشورمان در سالن حضور داشت و با گریه برای قهرمانی فرزندش دعا می‌کرد. امیرقاسمی منجزی فرنگی‌کار وزن 130 کیلوگرم در خصوص قهرمانی در جام تختی و همچنین حضور مادرش برای تماشای مبارزه فینال، عنوان کرد: خدا را شکر می‌کنم مسابقات تمام شد؛ واقعا یکی از سخت‌ترین کشتی‌های زندگی‌ام را در مبارزه فینال انجام دادم، چرا که نمی‌دانستم به فشار و استرس مبارزه فینال غلبه کنم یا اینکه مادرم آن بالا اتفاقی برایش رخ ندهد و پَس نیافتد. دارنده 2 مدال طلای قهرمانی آسیا ادامه داد: مسابقات جام تختی در سطح بالایی برگزار شد و خیلی سخت بود. خدا را شکر توانستم حداقل در این شرایط بد

In [62]:
query_text = "پرو"
similarity_function = 'cosine'
k = 10
query1 = query(query_text, similarity_function, k)

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 433267.84it/s]

Execution Time: 0.0310 seconds
Top 10 documents most relevant to the query:

Rank 1
Document ID: 10999
Similarity Score: 0.1723
بیانیه فراکسیون ایثارگری مجلس: به فیاضی رای می‌دهیم
https://www.farsnews.ir/news/14000825000074/بیانیه-فراکسیون-ایثارگری-مجلس-به-فیاضی-رای-می‌دهیم


Rank 2
Document ID: 9541
Similarity Score: 0.1572
تأکید بر تزریق دوز یادآور واکسن کرونا برای مقابله با سویه اومیکرون
https://www.farsnews.ir/news/14001001000753/تأکید-بر-تزریق-دوز-یادآور-واکسن-کرونا-برای-مقابله-با-سویه-اومیکرون


Rank 3
Document ID: 1541
Similarity Score: 0.1539
تور جهانی تنیس برزیل| پیروزی دختر تاریخ ساز مقابل تنیسور میزبان
https://www.farsnews.ir/news/14001204000315/تور-جهانی-تنیس-برزیل|-پیروزی-دختر-تاریخ-ساز-مقابل-تنیسور-میزبان
به گزارش خبرگزاری فارس، جدول اصلی مسابقات تور جهانی زیر ۱۸ سال ( سطح یک دنیا)  به میزبانی برزیل آغاز  شد و صفی به عنوان تنها نماینده کشورمان در بخش انفرادی به مصاف تنیسوری از کشور میزبان رفت.  در راند نخست جدول اصلی بخش انفرادی ، مشکات الزهرا صفی با نتیجه (۱-۶ ، ۶-۲ و ۰-

In [63]:
query_text = "برزیل ارژانتین"
similarity_function = 'cosine'
k = 10
query1 = query(query_text, similarity_function, k)

Calculating Similarity: 100%|██████████| 12202/12202 [00:00<00:00, 752221.55it/s]

Execution Time: 0.0187 seconds
Top 10 documents most relevant to the query:

Rank 1
Document ID: 3342
Similarity Score: 0.2298
آغاز سومین اردوی المپیکی تیم ملی والیبال ساحلی ناشنوایان در قشم
https://www.farsnews.ir/news/14001110000291/آغاز-سومین-اردوی-المپیکی-تیم-ملی-والیبال-ساحلی-ناشنوایان-در-قشم


Rank 2
Document ID: 10839
Similarity Score: 0.2274
نادری: اتاق بازرگانی ایران و برزیل به منظور گسترش روابط اقتصادی دو کشور افتتاح شد
https://www.farsnews.ir/news/14000828000423/نادری- اتاق بازرگانی ایران و برزیل به


Rank 3
Document ID: 2459
Similarity Score: 0.2186
رده بندی جدید فیفا | ایران همچنان در قله آسیا +عکس
https://www.farsnews.ir/news/14001121000322/رده-بندی-جدید-فیفا-|-ایران-همچنان-در-قله-آسیا-عکس
به گزارش خبرگزاری فارس، در جدید ترین رده بندی فیفا، تیم ملی فوتبال کشورمان بدون تغییر در جایگاه بیست و یکم دنیا قرار گرفت. تیم ملی فوتبال کشورمان در این دوره از بازی‌های ملی مقابل امارات و عراق به برتری رسید و توانست امتیازهای خود را به عدد 1572.89 برساند. ایران همچنان در رده اول آسیا ق

# Evaluation

## Memory usage

In [37]:
import sys

def size_of_obj(obj):
    return sys.getsizeof(obj)

# Usage
size_mb = size_of_obj(inverted_index) / (1024 * 1024)
print(f"Total memory usage of dictionary part: {size_mb:.2f} MB")

Total memory usage of dictionary part: 2.50 MB


## High & low idf

In [38]:
from heapq import nlargest, nsmallest

def find_extreme_idf_terms(doc_freq, N, top_k=3):
    idf_values = {term: math.log(N / freq, 10) for term, freq in doc_freq.items()}

    # Find the terms with the highest IDF values
    top_terms = nlargest(top_k, idf_values, key=idf_values.get)

    # Find the terms with the lowest IDF values
    bottom_terms = nsmallest(top_k, idf_values, key=idf_values.get)

    return top_terms, bottom_terms

In [39]:
top_terms, bottom_terms = find_extreme_idf_terms(doc_freq, N)
print("Top 3 terms with highest IDF:", top_terms)
print("Bottom 3 terms with lowest IDF:", bottom_terms)

Top 3 terms with highest IDF: ['۸اسلوونی', '۴فرانسه', '۱۴تونس']
Bottom 3 terms with lowest IDF: ['کرد&کن', 'خبرنگار', 'شد&شو']


## Saving with pickle

In [45]:
import pickle

# Assuming all your data structures are already filled with the data

# Save all objects into one file
with open('data_structures.pkl', 'wb') as file:
    pickle.dump({
        'tokenized': tokenized,
        'removed_tokenized': removed_tokenized,
        'stemmed_tokenized': stemmed_tokenized,
        'inverted_index': inverted_index,
        'champion_lists': champion_lists,
        'vectorized_docs': vectorized_docs,
        'weights_docs': weights_docs,
        'doc_freq': doc_freq,
        'N': N,
        'vectorized_champ': vectorized_champ,
        'weights_champ': weights_champ,
        'champ_freq': champ_freq,
        'N_champ': N_champ
    }, file)

print('All data structures have been saved to data_structures.pkl')

All data structures have been saved to data_structures.pkl


## Loading with pickle

In [46]:
import pickle

# Load all objects from the pickle file
with open('data_structures.pkl', 'rb') as file:
    data_structures = pickle.load(file)

# Extracting each data structure
tokenized = data_structures['tokenized']
removed_tokenized = data_structures['removed_tokenized']
stemmed_tokenized = data_structures['stemmed_tokenized']
inverted_index = data_structures['inverted_index']
champion_lists = data_structures['champion_lists']
vectorized_docs = data_structures['vectorized_docs']
weights_docs = data_structures['weights_docs']
doc_freq = data_structures['doc_freq']
N = data_structures['N']
vectorized_champ = data_structures['vectorized_champ']
weights_champ = data_structures['weights_champ']
champ_freq = data_structures['champ_freq']
N_champ = data_structures['N_champ']

print('All data structures have been loaded from data_structures.pkl')

All data structures have been loaded from data_structures.pkl
